# old

In [ ]:
import os
import numpy as np
from PIL import Image
import astra
import matplotlib.pyplot as plt

tiff_dir = './projections/'
tiff_files = sorted([f for f in os.listdir(tiff_dir) if f.endswith('.tif')])
projections = np.stack([np.array(Image.open(os.path.join(tiff_dir, f)), dtype=np.float32) for f in tiff_files])

num_projections, num_rows, num_cols = projections.shape
angles = np.linspace(0, np.pi, num_projections, endpoint=False)


In [ ]:
reconstructed_slices = []

for i in range(num_rows):
    # Get sinogram for slice i
    sinogram = projections[:, i, :]

    # Define geometry for this slice
    proj_geom = astra.create_proj_geom('parallel', 1.0, num_cols, angles)
    vol_geom = astra.create_vol_geom(num_cols, num_cols)

    # Create data objects
    sinogram_id = astra.data2d.create('-sino', proj_geom, sinogram)
    rec_id = astra.data2d.create('-vol', vol_geom)

    # Configure FBP algorithm
    cfg = astra.astra_dict('FBP_CUDA')
    cfg['ProjectionDataId'] = sinogram_id
    cfg['ReconstructionDataId'] = rec_id

    alg_id = astra.algorithm.create(cfg)
    astra.algorithm.run(alg_id)

    # Get reconstructed slice
    rec = astra.data2d.get(rec_id)
    reconstructed_slices.append(rec)

    # Clean up
    astra.algorithm.delete(alg_id)
    astra.data2d.delete([sinogram_id, rec_id])

# Convert to 3D volume
reconstructed_volume = np.stack(reconstructed_slices, axis=0)


# new

In [ ]:
import os
import numpy as np
import pydicom
import astra
from tqdm import tqdm

# Directory containing DICOM projection images
dcm_dir = 'C:/Users/fedhila/Desktop/2D Scans/scan_03/DICOM'
dcm_files = sorted([os.path.join(dcm_dir, f) for f in os.listdir(dcm_dir) if f.endswith('.dcm')])

# Load all DICOM projection images into a 3D NumPy array
print("Loading DICOM projection images...")
projections = np.stack([pydicom.dcmread(f).pixel_array.astype(np.float32) for f in tqdm(dcm_files)], axis=0)

# Get number of projections, rows, and cols
num_projections, num_rows, num_cols = projections.shape
print(f"Loaded {num_projections} projections, each with shape {num_rows} x {num_cols}")

# Define angles — assuming evenly spaced projections over 180° (π radians)
angles = np.linspace(0, np.pi, num_projections, endpoint=False)

# Optional: Check if CUDA is available (will be False for you)


# Reconstruct each slice
reconstructed_slices = []

print("Reconstructing slices with FBP (CPU)...")
for i in tqdm(range(num_rows)):
    # Extract the sinogram for slice i
    sinogram = projections[:, i, :]

    # Define geometry for this slice
    proj_geom = astra.create_proj_geom('parallel', 1.0, num_cols, angles)
    vol_geom = astra.create_vol_geom(num_cols, num_cols)

    # Create 2D data objects for CPU
    sinogram_id = astra.data2d.create('-sino', proj_geom, sinogram)
    rec_id = astra.data2d.create('-vol', vol_geom)

    # Configure the FBP algorithm for CPU
    cfg = astra.astra_dict('SIRT')
    cfg['ProjectionDataId'] = sinogram_id
    cfg['ReconstructionDataId'] = rec_id

    # Run the algorithm
    alg_id = astra.algorithm.create(cfg)
    astra.algorithm.run(alg_id)

    # Retrieve the reconstructed slice
    rec = astra.data2d.get(rec_id)
    reconstructed_slices.append(rec)

    # Clean up memory
    astra.algorithm.delete(alg_id)
    astra.data2d.delete([sinogram_id, rec_id])

# Convert list of 2D slices to 3D volume
reconstructed_volume = np.stack(reconstructed_slices, axis=0)

print(f"Reconstruction done. Volume shape: {reconstructed_volume.shape}")

# Visualization

In [ ]:
pip install --upgrade pyvista

In [ ]:
import os
import numpy as np
import imageio
import pyvista as pv
from tqdm import tqdm

# Corrected path to TIF slices folder
path = r"C:\Users\fedhila\Downloads\tif_slices-20250505T114141Z-001\tif_slices"

# List all tif files, sorted to maintain slice order
tif_files = sorted([os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith(('.tif', '.tiff'))])

# Load images into a 3D NumPy array
print("Loading slices...")
volume = np.stack([imageio.imread(f) for f in tqdm(tif_files)], axis=0)

print(f"Loaded volume shape: {volume.shape}")

# Convert to PyVista UniformGrid
nz, ny, nx = volume.shape
grid = pv.UniformGrid()

# Set the dimensions: shape + 1 because dimensions are points, not cells
grid.dimensions = (nx, ny, nz)

# Set the spacing (voxel size) — adjust if your voxels have physical spacing
grid.spacing = (1, 1, 1)

# Set the origin
grid.origin = (0, 0, 0)

# Add the volume data
grid.point_data["values"] = volume.flatten(order="F")  # Fortran order for 3D

# Visualize the volume
pl = pv.Plotter()
pl.add_volume(grid, cmap="bone", opacity="linear")
pl.show()
